# Bulk RNA-seq eQTL analysis

This notebook provide a master control on the XQTL workflow so it can works on multiple data collection as proposed.
Input:
    A recipe file,each row is a data collection and with the following column:
    
    Theme
        name of dataset, must be different, each uni_study analysis will be performed in a folder named after each, meta analysis will be performed in a folder named as {study1}_{study2}
            
        The column name must contain the # and be the first column
    
    genotype_list
        {Path to file}
    
    molecular_pheno
        {Path to file}
    region_list (list of regions to be analzed)
        {Path to file}
    
    covariate_file
        {Path to file}
    
    factor_analysis_opt
        "APEX" vs "PEER" for factor analysis
    
    LD options:
        "In-sample" LD vs {path to reference panel}
    
    QTL_tool_option
        "APEX" vs "TensorQTL" for QTL association
    
    QTL_analysis_option
        {Int for cis window} vs "trans"
    
    Populations
        The populations from which of the samples was drawn
    
    Conditions:
        The nature of molecular phenotype
    
    ### note: Only data collection from the same Populations and conditions will me merged to perform Fix effect meta analysis
    
Output:
    ...

## Generation of MWE
This is the code to generate the mwe recipe and LD_recipe on csg cluster

**FIXME: Hao, I'm thinking that for this configuration file we should not speicify any parameter (eg `cis_windows` below), but only file locations and workflow names. the outcome of running this should also be that instead of running anything, we write the sequence of commands to execute to a `sh` file (with automatic comment strings `#` to separate the sections) for people to run on their own, so they will have better control over everything. Is there an issue with this plan? (please slack me for discussions on this if necessary). Otherwise, let's focus first on finishing up the modules and worry about this the last?**

**FIXME: Also, this file should be those that reproduces exactly the official default Brain xQTL consortium analysis. That is why I would rather have it per xQTL type even if there will be duplicated lines of contents**

In [ ]:
Recipe_temp = pd.DataFrame( {"Theme" : ["AC","DLPFC","PCC"] ,
                "genotype_list" : ["/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_genotype_list",
                   "/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_genotype_list",
                   "/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_genotype_list"],
                "molecular_pheno" : ["/home/hs3163/Project/Rosmap/data/gene_exp/AC/geneTpmResidualsAgeGenderAdj_rename.txt",
                     "/home/hs3163/Project/Rosmap/data/gene_exp/DLPFC/geneTpmResidualsAgeGenderAdj_rename.txt",
                     "/home/hs3163/Project/Rosmap/data/gene_exp/PCC/geneTpmResidualsAgeGenderAdj_rename.txt"],
                "region_list" : ["~/GIT/ADSPFG-xQTL/MWE/mwe_region",
                "~/GIT/ADSPFG-xQTL/MWE/mwe_region",
                "~/GIT/ADSPFG-xQTL/MWE/mwe_region"] ,
                "covariate_file" : ["/home/hs3163/GIT/ADSPFG-xQTL/MWE/MWE.cov","/home/hs3163/GIT/ADSPFG-xQTL/MWE/MWE.cov","/home/hs3163/GIT/ADSPFG-xQTL/MWE/MWE.cov"],
                "factor_analysis_opt" : ["APEX","APEX","APEX"],
                "LD_Recipe": ["~/GIT/ADSPFG-xQTL/MWE/LD_Recipe","~/GIT/ADSPFG-xQTL/MWE/LD_Recipe","~/GIT/ADSPFG-xQTL/MWE/LD_Recipe"],
                "QTL_tool_option" : ["APEX","APEX","APEX"],
                "QTL_analysis_option" : ["cis","cis","cis"],
                "cis_windows" : [500000,500000,5000000],
                "Metal" : ["T","T","F"]}).to_csv("/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example","\t")

        ### note: Only data collection from the same Populations and conditions will me merged to perform Fix effect meta analysis
    

In [ ]:
pd.DataFrame({"ld_file_prefix" : ["/mnt/mfs/statgen/neuro-twas/mv_wg/cache_arch/cache/geneTpmResidualsAgeGenderAdj_rename.","/mnt/mfs/statgen/neuro-twas/mv_wg/cache_arch/cache/geneTpmResidualsAgeGenderAdj_rename."],
               "ld_file_surfix" : [".merged.ld.rds",".merged.ld.rds"]}).to_csv("~/GIT/ADSPFG-xQTL/MWE/LD_Recipe",sep = "\t")





## Example for running the workflow
This will run the workflow from via several submission and save the output to nohup.out

In [ ]:
nohup sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control.ipynb Fine_mapping \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow" -J 50 -q csg -c ~/GIT/ADSPFG-xQTL/code/csg.yml -s build &

## Other example workflow:
These command run each of the substep to test them individually

In [ ]:
sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control.ipynb data_proc \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow"

sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control.ipynb QTL \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow"

sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control.ipynb Meta \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow" 

sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control.ipynb Fine_mapping \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow" 




In [ ]:
[global]
## The aforementioned input recipe
parameter: recipe = path
## Overall wd, the file structure of analysis is wd/[steps]/[sub_dir for each steps]
parameter:  wd = path(".")
## Diretory to the excutable
parameter: exe_dir = path("~/GIT/ADSPFG-xQTL/workflow")

parameter: container = '/mnt/mfs/statgen/containers/twas_latest.sif'
parameter: container_lmm = '/mnt/mfs/statgen/containers/lmm.sif'
parameter: container_apex = '/mnt/mfs/statgen/containers/apex.sif'
parameter: container_gtex = '/mnt/mfs/statgen/containers/gtex_eqtl.sif'
parameter: yml = "/home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml"
import pandas as pd
input_inv = pd.read_csv(recipe, sep = "\t")
Metal_theme = input_inv.query("Metal == 'T'")["Theme"].to_list()
Metal_theme_str = "-".join(Metal_theme)
Non_Metal_theme = input_inv.query("Metal != 'T'")["Theme"].to_list()
Non_Metal_theme.append(Metal_theme_str)
Theme_Prefix = "_".join(Non_Metal_theme)
input_inv.assign(chrom = )
parameter: LD_Recipe  = path(input_inv["LD_Recipe"][0])
input_inv = input_inv.to_dict("records")


## Molecular Phenotype Calling

## Data Preprocessing
### Molecular Phenotype Processing

In [ ]:
[Normalization]
import os
input: for_each = "input_inv"
skip_if( os.path.exists(_input_inv["Molecular_Pheno_whole"]))
output:

bash:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/data_preprocessing/genotype/GWAS_QC.ipynb output \
        --genecount_table $[_input_inv["genecount_table"]] \
        --geneTpm_table $[_input_inv["geneTpm_table"]] \
        --sample_index $[_input_inv["sample_index"]] \
        --container $[container_gtex] \
        --wd $[wd:a]/data_preprocessing/genotype/qc/ \
        -J 200 -q csg -c $[yml]

### Genotype Processing

In [ ]:
[genotype_QC]
input: for_each = "input_inv"
output: per_chrom_plink_list = f'{wd:a}/data_preprocessing/genotype/qc/{_input_inv["Theme"]}.processed_genotype.plink_per_chrom.recipe',
        merged_plink =  f'{wd:a}/data_preprocessing/genotype/qc/{_input_inv["Theme"]}.mergrd.bed',
        unrelated = f'{wd:a}/data_preprocessing/genotype/qc/{_input_inv["Theme"]}.mergrd.unrelated.bed',
        related = f'{wd:a}/data_preprocessing/genotype/qc/{_input_inv["Theme"]}.mergrd.related.bed'
#task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/data_preprocessing/genotype/GWAS_QC.ipynb output \
        --genotype_list $[_input_inv["genotype_list"]] \
        --name $[_input_inv["Theme"]] \
        --container_lmm $[container_lmm] \
        --wd $[wd:a]/data_preprocessing/genotype/qc/ \
        -J 200 -q csg -c $[yml]

In [ ]:
[genotype_reformatting]
import pandas as pd
input: output_from("genotype_QC")["per_chrom_plink_list"], group_with = "input_inv"
output: vcf_list = f'{wd}/data_preprocessing/genotype/vcf/{_input_inv["Theme"]}.vcf.recipe',
        per_gene_plink = f'{wd}/data_preprocessing/genotype/per_gene/{_input_inv["Theme"]}.per_gene_plink.recipe'
#task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/data_preprocessing/genotype/genotype_formatting.ipynb plink2vcf \
        --genotype_list $[_input] \
        --name $[_input_inv["Theme"]] \
        --container_lmm $[container_lmm] \
        --wd $[wd:a]/data_preprocessing/genotype/vcf/ \
        -J 200 -q csg -c /home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml

     sos run $[exe_dir]/data_preprocessing/genotype/genotype_formatting.ipynb plink_by_gene \
        --genotype_list $[_input] \
        --name $[_input_inv["Theme"]] \
        --region_list $[_input_inv["region_list"]] \
        --container_lmm $[container_lmm] \
        --wd $[wd:a]/data_preprocessing/genotype/ \
        -J 200 -q csg -c $[yml]

In [ ]:
[LD]
import pandas as pd
input: output_from("genotype_reformatting")["per_gene_plink"],group_with = "input_inv"
output: f'{wd}/data_preprocessing/genotype/LD/{_input_inv["Theme"]}._LD_recipe'
#task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
     sos run $[exe_dir]/data_preprocessing/genotype/LD.ipynb LD \
        --genotype_list $[_input] \
        --name $[_input_inv["Theme"]] \
        --region_list $[_input_inv["region_list"]] \
        --container_lmm $[container_lmm] \
        --wd $[wd:a]/data_preprocessing/genotype/LD/ \
        -J 200 -q csg -c $[yml]

In [ ]:
[LD_Recipe]
input: output_from("LD"), group_by = "all"
output: f'{wd:a}/data_preprocessing/genotype/LD/sumstat_list'
python: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    import pandas as pd
    input_list = [$[_input:r,]]
    ld_recipe = pd.read_csv(input_list[0],sep = "\t")
    for x in range(1,len(input_list)):
        ld_recipe = ld_recipe.append(pd.read_csv(input_list[x],sep = "\t"))
    ld_recipe.to_csv("$[_output]", index = 0 , sep = "\t")

In [ ]:
[GRM]
import pandas as pd
input: output_from("genotype_QC")["per_chrom_plink_list"],group_with = "input_inv"
output: f'{wd}/data_preprocessing/genotype/grm/{_input_inv["Theme"]}.grm_list.txt'
#task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
     sos run $[exe_dir]/data_preprocessing/genotype/GRM.ipynb GRM \
        --genotype_list $[_input] \
        --name $[_input_inv["Theme"]] \
        --container $[container_apex] \
        --wd $[wd:a]/data_preprocessing/genotype/grm/ \
        -J 200 -q csg -c $[yml]

## Factor analysis

In [ ]:
[factor]
input: output_from("genotype_reformatting")["vcf_list"],output_from("phenotype_reformating") ,group_with = "input_inv"
output: f'{wd}/data_Processing/covariate/{_input_inv["factor_analysis_opt"]}/{_input_inv["Theme"]}.factor.cov'
#task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/data_preprocessing/covariate/$[_input_inv["factor_analysis_opt"]].ipynb $[_input_inv["factor_analysis_opt"]] \
        --molecular_pheno_whole $[_input[1]] \
        --region_list $[_input_inv["region_list"]] \
        --genotype $[_input[0]] \
        --name $[_input_inv["Theme"]] \
        --wd $[wd:a]/data_preprocessing/covariate/ \
        -J 200 -q csg -c /home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml $[f'--covariate {_input_inv["covariate_file"]}' if os.path.exists(covariate) else f'']

In [ ]:
[pca]
import pandas as pd
input: output_from("genotype_QC")["related"],output_from("genotype_QC")["unrelated"],output_from("phenotype_reformating"),group_with = "input_inv"
output: f'{wd}/data_preprocessing/covariate/pca/{_input_inv["Theme"]}.pca.cov'
#task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
     sos run $[exe_dir]/data_preprocessing/covariate/PCA.ipynb PCA \
        --related $[_input[0]] \
        --unrelated $[_input[1]] \
        --name $[_input_inv["Theme"]] \
        --molecular_phenotype $[_input[2]]
        --container $[container_lmm] \
        --wd $[wd:a]/data_preprocessing/covariate/pca/ \
        -J 200 -q csg -c /home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml

## QTL associations

In [ ]:
[QTL_1]
input: output_from("pca_factor_merge"),output_from("grm"),output_from("phenotype_reformatting")["per_chrom_pheno_list"],output_from("genotype_reformatting")["vcf_list"],group_with = "input_inv"
output: f'{wd:a}/association_scan/{_input_inv["QTL_tool_option"]}/{_input_inv["QTL_analysis_option"]}/{_input_inv["Theme"]}.QTL_output_recipe.tsv'
#task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/association_scan/$[_input_inv["QTL_tool_option"]]/$[_input_inv["QTL_tool_option"]].ipynb $[_input_inv["QTL_analysis_option"]] \
        --molecular_pheno $[_input[2]] \
        --covariate $[_input[0]]\
        --genotype_list $[_input[3]] \
        --container $[container_apex] \
        --window $[_input_inv["cis_windows"]] \
        --name $[_input_inv["Theme"]]  \
        --wd $[wd:a]/QTL_association/ \       
        -J 200 -q csg -c /home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml $[f'--grm {_input[1]}' if $[_input_inv["QTL_tool_option"]] == "APEX" else f'']

#### Example:

sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/QTL_association/QTL_association.ipynb APEX_cis_Recipe \
    --recipe Data_Processing/PCC.data_proc_output_recipe.tsv \
    --container /mnt/mfs/statgen/containers/apex.sif \
    --window 500000 \
    --name PCC  \
    --wd /mnt/mfs/statgen/xqtl_workflow_testing/testing_no_cov/QTL_association/ \
    -J 200 -q csg -c /home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml

In [ ]:
[QTL_2]
input: group_by = "all"  
output: f'{_input[0]:d}/sumstat_list'
python: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    import pandas as pd
    input_list = [$[_input:r,]]
    input_inv = $[input_inv]
    chr_temp = [int(x[0].split(".")[-5].replace("chr","")) for x in  pd.read_csv(input_list[0]).values.tolist()]
    sumstat_list = pd.DataFrame({"chr" : chr_temp })
    for x in range(0,len(input_list)):
        sumstat_list = sumstat_list.assign(**{input_inv[x]["Theme"] : pd.read_csv(input_list[x])} )
    sumstat_list.to_csv("$[_output]", index = 0 , sep = "\t")

## Meta Analysis
Input: 
1. A recipe generated from the combination of previouse steps

Output: 
1. Recipe for Prior, Vhat, rds input, resid corr
3. vcf

In [ ]:
[Meta]
input: output_from("QTL_2")
output: f'{wd}/Meta_analysis/Fine_mapping_recipe.txt'
#task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/Meta_Analysis/Meta_Analysis.ipynb Recipe \
        --sumstat_list_path $[_input:a] \
        --region_list $[input_inv[0]["region_list"]] \
        --container $[container] \
        --wd $[wd:a]/Meta_analysis/ \
        --METAL_theme_str $[",".join(Metal_theme)] \
        -J 200 -q csg -c /home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml

#### Example
sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/Meta_Analysis/Meta_Analysis.ipynb Recipe \
    --sumstat_list_path /mnt/mfs/statgen/xqtl_workflow_testing/testing_no_cov/QTL_association/sumstat_list \
    --region_list ~/GIT/ADSPFG-xQTL/MWE/mwe_region_long \
    --container /mnt/mfs/statgen/containers/twas_latest.sif \
    --wd /mnt/mfs/statgen/xqtl_workflow_testing/testing_no_cov/Meta_analysis/ \
    --METAL_theme_str PCC,DLPFC \
    -J 200 -q csg -c /home/hs3163/GIT/ADSPFG-xQTL/code/csg.yml

## Fine mapping
Input: 
1. Recipe from previous step
2. a Recipe for LD, need to be made seperately

Output: vcf

In [ ]:
[Fine_mapping]
input: output_from("Meta")
output: f'{wd}/Fine_Mapping/{Theme_Prefix}.unisusie_rss.output_list.txt',
        f'{wd}/Fine_Mapping/{Theme_Prefix}.mvsusie_rss.output_list.txt'
#task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    sos run $[exe_dir]/Fine_Mapping/SumStat/SuSiE_RSS.ipynb Fine_mapping \
        --recipe $[_input:a] \
        --LD_Recipe $[LD_Recipe:a] \
        --container $[container] \
        --wd $[wd:a]/Fine_Mapping/ \
        --Theme_prefix $[Theme_Prefix] # \
        #-J 50 -q csg -c ~/GIT/ADSPFG-xQTL/code/csg.yml &

#### Example
sos run /home/hs3163/GIT/ADSPFG-xQTL/workflow/Fine_Mapping/SumStat/SuSiE_RSS.ipynb Fine_mapping \
    --recipe /mnt/mfs/statgen/xqtl_workflow_testing/testing_no_cov/Meta_analysis/Fine_mapping_recipe.txt \
    --LD_Recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/LD_Recipe \
    --container /mnt/mfs/statgen/containers/twas_latest.sif \
    --wd /mnt/mfs/statgen/xqtl_workflow_testing/testing_no_cov/Fine_Mapping/ \
    --Theme_prefix AC_PCC-DLPFC # \
    # -J 50 -q csg -c ~/GIT/ADSPFG-xQTL/code/csg.yml &